# Mount Gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


GPU info

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb 14 07:26:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Get COTS dataset

In [3]:
!cp /content/drive/MyDrive/'Colab Notebooks'/cots/kaggle.com_cookies.txt .
!mkdir tensorflow-great-barrier-reef
!wget -x --load-cookies kaggle.com_cookies.txt "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/31703/2871752/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1645007547&Signature=iU%2BAlMdGOeDlwesJs7EDm39wcJIYVPy8Zs9wumE25F%2FruKm%2Fng591v9lMr%2B8FFXKnlHU2aD07%2BVBQVSaA0MMNTXW%2Ffy2wUXqs4P59Ib99n%2Fjqa8h8aF4RshE8ozdwQxrkxZN24Q%2FtRwZ8OExMohvFrcFlIGfCQsUW%2B8FOVQDOfhhWqMt6V0zxDmwmxjxMxCf4aAWO0%2BhxC8zIICZEN57syTmiNXGMsXlBYK%2BakIRrFLw6ba4n8GWUIFJ0UyWR%2FaFSbAVm6yF%2Fh2ukiTUYUNdvZfI7FuML%2Fix4uWz5Md%2FF8LQVLfI1C3Haxe0%2BkQilweynJu8V1nG1AKvQcvyIj4cFw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtensorflow-great-barrier-reef.zip" -O data.zip
!unzip data.zip -d tensorflow-great-barrier-reef
!rm data.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4303.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4304.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4305.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4306.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4307.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4308.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4309.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/431.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4310.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4311.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4312.jpg  
  inflating: tensorflow-great-barrier-reef/train_images/video_2/4313.jpg  
  inflating: tensorflow-great-bar

# Import libraries

In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import ast
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GroupKFold

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Train


## Load the competition data and create temp directories

In [20]:
%cd /content
fold = 1

train = pd.read_csv('tensorflow-great-barrier-reef/train.csv')

!rm -rf yolo_data

!mkdir -p ./yolo_data/fold{fold}/images/val
!mkdir -p ./yolo_data/fold{fold}/images/train

!mkdir -p ./yolo_data/fold{fold}/labels/val
!mkdir -p ./yolo_data/fold{fold}/labels/train

train['num_bbox'] = train.annotations.apply(lambda x: len(ast.literal_eval(x)))

/content


## Add 20% of empty images for each sequence

In [21]:
# For each sequence get only rows with annotations but also add X% of rows without annotations
empty_photos_percent = 10
sequences = train.sequence.unique()
indexes_to_save = list()

# For each sequence get indexes of rows without annotations to save in dataset
for i in range(len(sequences)):
    sequence_len = len(train[(train.sequence == sequences[i])])
    empty_indexes = list(train[(train.sequence == sequences[i]) & (train.num_bbox == 0)].index)
    non_empty_indexes_len = sequence_len - len(empty_indexes)
    if len(empty_indexes) == 0 or non_empty_indexes_len == 0:
        continue
    np.random.seed(42)
    indexes_to_save.extend(np.random.choice(empty_indexes, int(non_empty_indexes_len*(empty_photos_percent)/100)+1, replace=False))

## Add fold number to each dataset's row

In [22]:
train = train[(train.num_bbox > 0) | (train.video_id == fold) | (train.index.isin(indexes_to_save))]
train.groupby('video_id').agg({'num_bbox': 'count'})

num_bbox
video_id          
0             2362
1             8232
2              746

In [23]:
# train = train.reset_index(drop=True)
# train['index'] = train.index

# fold_num = 10
# fold_size = int(train.shape[0]/10)+1

# train['fold'] = train.apply(lambda x: int(x['index']/fold_size), axis=1)

# train.head()

## Make annotations

In [24]:
annos = []
for i, row in tqdm(train.iterrows(), total = len(train)):
    if row.video_id == fold:
        # val
        mode = 'val'
    else:
        # train
        mode = 'train'
        # if x.num_bbox == 0: 
        #     continue
    copyfile(f'tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{row.image_id}.jpg')
    if row.num_bbox == 0:
        continue
    r = ''
    anno = eval(row.annotations)
    # fix corrupted annotation
    if row.image_id == '0-9470':
      anno[0]['height'] = 16
    for idx, an in enumerate(anno):
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{row.image_id}.txt', 'w') as fp:
        fp.write(r)

  0%|          | 0/11340 [00:00<?, ?it/s]

## Set hyperparameters and data settings


In [33]:
hyps = '''
# YOLOv5 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials


lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005 # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.2  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.5  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.5  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
'''

data = f'''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /content/yolo_data/fold{fold}/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''



## Get YOLOv5 and necessary libraries and set it's parameters 

In [26]:
# !pip install albumentations==1.0.3 
# !pip uninstall -y opencv-python-headless==4.5.5.62 
# !pip install opencv-python-headless==4.5.2.52

In [27]:
!pip install wandb
!rm -r /content/yolov5
!git clone https://github.com/ultralytics/yolov5 # clone
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check


YOLOv5 🚀 v6.0-253-ga45e472 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (4 CPUs, 25.5 GB RAM, 63.5/166.8 GB disk)


In [34]:
with open('/content/yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('/content/yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

Fix YOLOv5 files to measure F2 metrics

In [29]:
!cp -f /content/drive/MyDrive/'Colab Notebooks'/cots/yolov5_f2_metric/val.py /content/yolov5
!cp -f /content/drive/MyDrive/'Colab Notebooks'/cots/yolov5_f2_metric/metrics.py /content/yolov5/utils
!cp -f /content/drive/MyDrive/'Colab Notebooks'/cots/yolov5_f2_metric/__init__.py /content/yolov5/utils/loggers

Download the best model on the disk and train

In [36]:
!python train.py --project /content/drive/MyDrive/'Colab Notebooks'/cots/ --img 1920 --batch 8 --epochs 25 --optimizer Adam --data data/reef_f1_naive.yaml --weights yolov5s6.pt --name yolov5s6_1920_batch_8_no_anns_10_adam_video_fold{fold} --hyp data/hyps/hyp.heavy.2.yaml

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
train: weights=yolov5s6.pt, cfg=, data=data/reef_f1_naive.yaml, hyp=data/hyps/hyp.heavy.2.yaml, epochs=25, batch_size=8, imgsz=1920, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=/content/drive/MyDrive/Colab Notebooks/cots/, name=yolov5s6_1920_batch_8_no_anns_10_adam_video_fold1, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_peri

In [31]:
# from google.colab import files
# files.download("output/model_best.pth")

This should prevent Colab from idle disconnect.
Paste this into the JavaScript console of the Colab browser window (Ctrl+Shift+i)

In [32]:
# function ClickConnect() {
#   console.log('Working')
#   document
#     .querySelector('#top-toolbar > colab-connect-button')
#     .shadowRoot.querySelector('#connect')
#     .click()
# }

# setInterval(ClickConnect, 1800000)